In [1]:
import pandas as pd # pandas dataframes
from sklearn.ensemble import RandomForestClassifier # model that takes in data to train and classify the imports
from sklearn.metrics import f1_score, make_scorer, classification_report 
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import LabelEncoder, OneHotEncoder # one hot encoding of team names
import numpy as np
from collections import defaultdict
import time


In [2]:
# import csv files from git repository
teamPath = '/Users/lorneez/projects/sports_predictor/Sports_Model/Data/teamData.csv'
playerPath = '/Users/lorneez/projects/sports_predictor/Sports_Model/Data/playerData.csv'
outcomePath = '/Users/lorneez/projects/sports_predictor/Sports_Model/Data/outcomeData.csv'

In [3]:
# convert csv files into pandas dataframes
teamData = pd.read_csv(teamPath)
playerData = pd.read_csv(playerPath, engine='python')
outcomeData = pd.read_csv(outcomePath)

In [5]:
for col in teamData.columns:
    print(col)

Team
Year
G_PG
MP_PG
FG_PG
FGA_PG
FG%_PG
3P_PG
3PA_PG
3P%_PG
2P_PG
2PA_PG
2P%_PG
FT_PG
FTA_PG
FT%_PG
ORB_PG
DRB_PG
TRB_PG
AST_PG
STL_PG
BLK_PG
TOV_PG
PF_PG
PTS_PG
G_PP
MP_PP
FG_PP
FGA_PP
FG%_PP
3P_PP
3PA_PP
3P%_PP
2P_PP
2PA_PP
2P%_PP
FT_PP
FTA_PP
FT%_PP
ORB_PP
DRB_PP
TRB_PP
AST_PP
STL_PP
BLK_PP
TOV_PP
PF_PP
PTS_PP
G_OppPG
MP_OppPG
FG_OppPG
FGA_OppPG
FG%_OppPG
3P_OppPG
3PA_OppPG
3P%_OppPG
2P_OppPG
2PA_OppPG
2P%_OppPG
FT_OppPG
FTA_OppPG
FT%_OppPG
ORB_OppPG
DRB_OppPG
TRB_OppPG
AST_OppPG
STL_OppPG
BLK_OppPG
TOV_OppPG
PF_OppPG
PTS_OppPG
G_OppPP
MP_OppPP
FG_OppPP
FGA_OppPP
FG%_OppPP
3P_OppPP
3PA_OppPP
3P%_OppPP
2P_OppPP
2PA_OppPP
2P%_OppPP
FT_OppPP
FTA_OppPP
FT%_OppPP
ORB_OppPP
DRB_OppPP
TRB_OppPP
AST_OppPP
STL_OppPP
BLK_OppPP
TOV_OppPP
PF_OppPP
PTS_OppPP
Age
W
L
PW
PL
MOV
SOS
SRS
ORtg
DRtg
NRtg
Pace
FTr
3PAr
TS%
eFG%
TOV%
ORB%
FT/FGA
eFG%.1
TOV%.1
DRB%
FT/FGA.1
Arena
Attend.
Attend./G


### Format All Data

In [6]:
# drop unwanted columns
outcomeData = outcomeData.drop(["Start Time", "Box Score", "Notes"], axis=1)

# change OT to integer value for number of OT rounds
outcomeData.loc[(outcomeData["OT?"].isna()),'OT?']= 0
outcomeData.loc[(outcomeData["OT?"] == "OT"),'OT?']= 1
outcomeData.loc[(outcomeData["OT?"] == "2OT"),'OT?']= 2
outcomeData.loc[(outcomeData["OT?"] == "3OT"),'OT?']= 3
outcomeData.loc[(outcomeData["OT?"] == "4OT"),'OT?']= 4


In [7]:
# function for redefining team names
def redefineTeamNames(dataframe):
    
    dataframe = dataframe.replace('Seattle SuperSonics', 'Oklahoma City Thunder', regex=True)
    
    dataframe = dataframe.replace('New Orleans/Oklahoma City Hornets', 'New Orleans Pelicans', regex=True)

    dataframe = dataframe.replace('New Orleans Hornets', 'New Orleans Pelicans', regex=True)
    
    dataframe = dataframe.replace('Charlotte Bobcats', 'Charlotte Hornets', regex=True)
    
    dataframe = dataframe.replace('New Jersey Nets', 'Brooklyn Nets', regex=True)

    return dataframe

In [8]:
# call function to redefine team names
outcomeData = redefineTeamNames(outcomeData)
teamData = redefineTeamNames(teamData)

# set up encoder and use one hot encoding to transform team names
encoding = LabelEncoder()
encoding.fit(outcomeData['Home'].values)
outcomeData['HomeEncoded'] = encoding.transform(outcomeData['Home'].values)
outcomeData['VisitorEncoded'] = encoding.transform(outcomeData['Visitor'].values)

In [9]:
# generate column names
teamDataColumn = teamData.drop(['Team', "Year", "Arena", "Attend.", "Attend./G", "G_PG", "MP_PG"], axis=1).columns
print(teamDataColumn)
testTeamDataColumn = teamData.drop(teamData.columns.difference(["3P_PG", "FGA_PG", "FG%_PG", "3P_PG", 
                                                               "3PA_PG", "3P%_PG", "ORB_PG", "DRB_PG", "TRB_PG", "AST_PG"
                                                                , "STL_PG", "BLK_PG", "TOV_PG", "2P_PG", "2PA_PG"]), axis=1).columns
print(testTeamDataColumn)
# outcomeData = outcomeData.loc[outcomeData['Year'] == 2020]
outcomeData



Index(['FG_PG', 'FGA_PG', 'FG%_PG', '3P_PG', '3PA_PG', '3P%_PG', '2P_PG',
       '2PA_PG', '2P%_PG', 'FT_PG',
       ...
       '3PAr', 'TS%', 'eFG%', 'TOV%', 'ORB%', 'FT/FGA', 'eFG%.1', 'TOV%.1',
       'DRB%', 'FT/FGA.1'],
      dtype='object', length=113)
Index(['FGA_PG', 'FG%_PG', '3P_PG', '3PA_PG', '3P%_PG', '2P_PG', '2PA_PG',
       'ORB_PG', 'DRB_PG', 'TRB_PG', 'AST_PG', 'STL_PG', 'BLK_PG', 'TOV_PG'],
      dtype='object')


,Year,Month,Visitor,VisitorPTS,Home,HomePTS,OT?,Attend,Total Points,Winner,HomeEncoded,VisitorEncoded
0,2005,november,Houston Rockets,79,Detroit Pistons,87,0,22076,166,Detroit Pistons,8,10
1,2005,november,Sacramento Kings,98,Dallas Mavericks,107,0,20041,205,Dallas Mavericks,6,25
2,2005,november,Denver Nuggets,78,Los Angeles Lakers,89,0,18997,167,Los Angeles Lakers,13,7
3,2005,november,Indiana Pacers,109,Cleveland Cavaliers,104,2,19730,213,Indiana Pacers,5,11
4,2005,november,Milwaukee Bucks,92,Orlando Magic,93,0,15138,185,Orlando Magic,21,16
...,...,...,...,...,...,...,...,...,...,...,...,...
20429,2020,march,Los Angeles Clippers,131,Golden State Warriors,107,0,18064,238,Los Angeles Clippers,9,12
20430,2020,march,Detroit Pistons,106,Philadelphia 76ers,124,0,20172,230,Philadelphia 76ers,22,8
20431,2020,march,New York Knicks,136,Atlanta Hawks,131,1,15393,267,New York Knicks,0,19
20432,2020,march,Charlotte Hornets,109,Miami Heat,98,0,19600,207,Charlotte Hornets,15,3


### Feature Engineering

#### y data

In [10]:
# function for finding if the home team is the winner
def homeWin(dataframe):
    if dataframe['Winner'] == dataframe['Home']:
        return True
    else:
        return False
    
# add in HomeWin column
outcomeData['HomeWin'] = outcomeData.apply(homeWin, axis=1)

In [11]:
won_last = defaultdict(int)

for index, row in outcomeData.iterrows():
    home_team = row["Home"]
    visitor_team = row["Visitor"]
    outcomeData.loc[index,'HomeLastWin'] = won_last[home_team]
    outcomeData.loc[index,'VisitorLastWin'] = won_last[visitor_team]
    won_last[home_team] = row["HomeWin"]
    won_last[visitor_team] = not row["HomeWin"]

KeyboardInterrupt: 

In [ ]:
outcomeData["HomeWinStreak"] = 0
outcomeData["VisitorWinStreak"] = 0

win_streak = defaultdict(int)

for index, row in outcomeData.iterrows():
    home_team = row["Home"]
    visitor_team = row["Visitor"]
    outcomeData.loc[index,'HomeWinStreak'] = win_streak[home_team]
    outcomeData.loc[index,'VisitorWinStreak'] = win_streak[visitor_team]
    if row["HomeWin"]:
        win_streak[home_team] += 1
        win_streak[visitor_team] = 0
    else:
        win_streak[home_team] = 0
        win_streak[visitor_team] += 1

In [12]:
outcomeData

,Year,Month,Visitor,VisitorPTS,Home,HomePTS,OT?,Attend,Total Points,Winner,HomeEncoded,VisitorEncoded,HomeWin,HomeLastWin,VisitorLastWin,HomeWinStreak,VisitorWinStreak
0,2005,november,Houston Rockets,79,Detroit Pistons,87,0,22076,166,Detroit Pistons,8,10,True,0,0,0,0
1,2005,november,Sacramento Kings,98,Dallas Mavericks,107,0,20041,205,Dallas Mavericks,6,25,True,0,0,0,0
2,2005,november,Denver Nuggets,78,Los Angeles Lakers,89,0,18997,167,Los Angeles Lakers,13,7,True,0,0,0,0
3,2005,november,Indiana Pacers,109,Cleveland Cavaliers,104,2,19730,213,Indiana Pacers,5,11,False,0,0,0,0
4,2005,november,Milwaukee Bucks,92,Orlando Magic,93,0,15138,185,Orlando Magic,21,16,True,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20429,2020,march,Los Angeles Clippers,131,Golden State Warriors,107,0,18064,238,Los Angeles Clippers,9,12,False,True,False,1,0
20430,2020,march,Detroit Pistons,106,Philadelphia 76ers,124,0,20172,230,Philadelphia 76ers,22,8,True,False,False,0,0
20431,2020,march,New York Knicks,136,Atlanta Hawks,131,1,15393,267,New York Knicks,0,19,False,True,False,1,0
20432,2020,march,Charlotte Hornets,109,Miami Heat,98,0,19600,207,Charlotte Hornets,15,3,False,True,False,1,0


In [ ]:
def home_team_ranks_higher(row):
    home_team = row["Home Team"]
    visitor_team = row["Visitor Team"]
    home_rank = ladder.loc[home_team]["Rk"]
    visitor_rank = ladder.loc[visitor_team]["Rk"]
    return home_rank < visitor_rank

In [ ]:
def home_team_won_last(row):
    home_team = row["Home Team"]
    visitor_team = row["Visitor Team"]
    
    teams = tuple(sorted([home_team, visitor_team]))
    result = 1 if last_match_winner[teams] == row["Home Team"] else 0
    
    winner = row["Home Team"] if row["HomeWin"] else row["Visitor Team"]
    last_match_winner[teams] = winner
    
    return result
    
outcomeData['HomeTeamWonLast'] = outcomeData.apply(home_team_won_last, axis=1)

In [ ]:
# spread
outcomeData["Final Home Spread"] = (outcomeData["VisitorPTS"] - outcomeData["HomePTS"]).astype(float)

#### x data

In [14]:
def differentialVariableGenerator(outcomeData, teamData, teamDataColumn):
                                  
    global teamData_global
    global counter 
    global differentialFeatureData
    x = []
    for col in teamDataColumn:
        x.append(col + "_home")
        x.append(col + "_visitor")
    
    differentialFeatureData = pd.DataFrame(index=range(outcomeData.shape[0]), columns = x)
    print(outcomeData.shape[0])
    teamData_global = teamData
    totalCount = outcomeData.shape[0]
    counter = 0
    overallTime = time.time()
    
    def calcDifferences(row):
                                  
        global teamData_global
        global counter
                                  
        counter = counter + 1;
        start = time.time()
        year = row['Year']
        visitor = row['Visitor']
        home = row['Home']
                                  
        def columnCalc(col):
                                  
            variable = col.name
            var_home = variable+"_home"
            var_visitor = variable+"_visitor"
            year_subset = teamData_global.loc[(teamData_global.Year == year)]
            visitorValue = year_subset.loc[year_subset.Team.astype(str).str.contains(visitor)][variable]
            homeValue = year_subset.loc[year_subset.Team.astype(str).str.contains(home)][variable]
            #difference = homeValue.iloc[0] - visitorValue.iloc[0]
            differentialFeatureData[var_home][counter-1] = homeValue
            differentialFeatureData[var_visitor][counter-1] = visitorValue

            #print(homeValue, visitorValue, difference)
                                  
        teamData_global[teamDataColumn].apply(columnCalc)
        print(visitor, home, time.time() - start,time.time() - overallTime)
        print("{count}/{total}".format(count = counter, total = totalCount))
                                  
    outcomeData.apply(calcDifferences, axis=1)
    print("Total time {time} seconds".format(time = time.time() - overallTime))
    return differentialFeatureData

In [15]:
testOutcomeData = differentialVariableGenerator(outcomeData, teamData, testTeamDataColumn)

20434
Houston Rockets Detroit Pistons 0.12844276428222656 0.1776728630065918
1/20434
Sacramento Kings Dallas Mavericks 0.09277796745300293 0.27065396308898926
2/20434
Denver Nuggets Los Angeles Lakers 0.0768439769744873 0.34769582748413086
3/20434
Indiana Pacers Cleveland Cavaliers 0.06750297546386719 0.4160459041595459
4/20434
Milwaukee Bucks Orlando Magic 0.10677599906921387 0.5230767726898193
5/20434
Houston Rockets Toronto Raptors 0.06235027313232422 0.5882329940795898
6/20434
Philadelphia 76ers Boston Celtics 0.05687999725341797 0.6454458236694336
7/20434
Washington Wizards Memphis Grizzlies 0.054039955139160156 0.6996719837188721
8/20434
New York Knicks Minnesota Timberwolves 0.07663488388061523 0.7765309810638428
9/20434
Miami Heat Brooklyn Nets 0.13620877265930176 0.912996768951416
10/20434
Dallas Mavericks New Orleans Pelicans 0.07768797874450684 0.990933895111084
11/20434
Sacramento Kings San Antonio Spurs 0.10957479476928711 1.100944995880127
12/20434
Atlanta Hawks Phoenix S

Phoenix Suns Dallas Mavericks 0.315471887588501 11.749369859695435
102/20434
New York Knicks San Antonio Spurs 0.28266191482543945 12.032435894012451
103/20434
Chicago Bulls Sacramento Kings 0.26376914978027344 12.296878814697266
104/20434
Toronto Raptors Los Angeles Clippers 0.43033409118652344 12.729766845703125
105/20434
Atlanta Hawks Indiana Pacers 0.43720197677612305 13.181208848953247
106/20434
Utah Jazz Orlando Magic 0.3529839515686035 13.534558057785034
107/20434
Boston Celtics Washington Wizards 0.12588810920715332 13.660701990127563
108/20434
Minnesota Timberwolves Detroit Pistons 0.2761809825897217 13.937180995941162
109/20434
Oklahoma City Thunder Brooklyn Nets 0.13519716262817383 14.072792053222656
110/20434
Phoenix Suns New Orleans Pelicans 0.18683481216430664 14.25987696647644
111/20434
Toronto Raptors Denver Nuggets 0.24610185623168945 14.506584882736206
112/20434
Miami Heat Milwaukee Bucks 0.0823519229888916 14.58930778503418
113/20434
Memphis Grizzlies Portland Trail 

Charlotte Hornets Brooklyn Nets 0.07665514945983887 29.900884866714478
204/20434
Sacramento Kings Memphis Grizzlies 0.11800813674926758 30.01977801322937
205/20434
Los Angeles Lakers Milwaukee Bucks 0.05938720703125 30.079551935195923
206/20434
San Antonio Spurs Dallas Mavericks 0.062470197677612305 30.142231941223145
207/20434
Detroit Pistons Houston Rockets 0.05869007110595703 30.201841831207275
208/20434
Phoenix Suns Utah Jazz 0.05388998985290527 30.255970001220703
209/20434
Oklahoma City Thunder Portland Trail Blazers 0.07379889488220215 30.330501794815063
210/20434
Milwaukee Bucks Boston Celtics 0.05950188636779785 30.390955924987793
211/20434
Toronto Raptors Orlando Magic 0.07952570915222168 30.47071599960327
212/20434
Brooklyn Nets Washington Wizards 0.052462100982666016 30.523838996887207
213/20434
Memphis Grizzlies New York Knicks 0.057624101638793945 30.581735849380493
214/20434
Sacramento Kings New Orleans Pelicans 0.07467293739318848 30.65661597251892
215/20434
Los Angeles 

Orlando Magic Phoenix Suns 0.06520390510559082 40.07046985626221
305/20434
Boston Celtics Los Angeles Clippers 0.05814504623413086 40.128798961639404
306/20434
New Orleans Pelicans Charlotte Hornets 0.06923127174377441 40.19828200340271
307/20434
Denver Nuggets Philadelphia 76ers 0.05749201774597168 40.25612282752991
308/20434
Indiana Pacers Atlanta Hawks 0.06703305244445801 40.32336688041687
309/20434
New York Knicks Brooklyn Nets 0.05663490295410156 40.38040089607239
310/20434
Sacramento Kings Milwaukee Bucks 0.07828307151794434 40.4589729309082
311/20434
Portland Trail Blazers Minnesota Timberwolves 0.08248090744018555 40.54285287857056
312/20434
Golden State Warriors Dallas Mavericks 0.06593680381774902 40.609113931655884
313/20434
Los Angeles Clippers Utah Jazz 0.07245683670043945 40.68183875083923
314/20434
Los Angeles Lakers Oklahoma City Thunder 0.07513809204101562 40.757208824157715
315/20434
Denver Nuggets Boston Celtics 0.07033109664916992 40.82777810096741
316/20434
Portlan

Phoenix Suns San Antonio Spurs 0.06629610061645508 48.51104688644409
405/20434
Philadelphia 76ers Oklahoma City Thunder 0.0595400333404541 48.57089376449585
406/20434
Toronto Raptors Los Angeles Lakers 0.15289711952209473 48.72408390045166
407/20434
Indiana Pacers Charlotte Hornets 0.09987592697143555 48.82429003715515
408/20434
Houston Rockets Cleveland Cavaliers 0.08282303810119629 48.908201932907104
409/20434
Milwaukee Bucks Orlando Magic 0.06458115577697754 48.972962856292725
410/20434
Detroit Pistons Washington Wizards 0.24239611625671387 49.21574783325195
411/20434
Minnesota Timberwolves New York Knicks 0.08356618881225586 49.301175117492676
412/20434
Boston Celtics Memphis Grizzlies 0.07184195518493652 49.37340688705444
413/20434
Phoenix Suns New Orleans Pelicans 0.06402587890625 49.43767976760864
414/20434
Toronto Raptors Golden State Warriors 0.06323790550231934 49.501189947128296
415/20434
Utah Jazz Los Angeles Clippers 0.06821584701538086 49.57034087181091
416/20434
Oklahoma

...

Portland Trail Blazers Toronto Raptors 0.10983800888061523 4449.067748785019
20012/20434
Oklahoma City Thunder Brooklyn Nets 0.1597888469696045 4449.235139131546
20013/20434
Minnesota Timberwolves Memphis Grizzlies 0.09949493408203125 4449.3349549770355
20014/20434
Sacramento Kings Phoenix Suns 0.10475015640258789 4449.439900875092
20015/20434
New York Knicks Los Angeles Lakers 0.09244799613952637 4449.5327570438385
20016/20434
San Antonio Spurs Boston Celtics 0.14371585845947266 4449.676687955856
20017/20434
Toronto Raptors Charlotte Hornets 0.19278311729431152 4449.870563745499
20018/20434
Miami Heat Indiana Pacers 0.164139986038208 4450.036252975464
20019/20434
Washington Wizards Orlando Magic 0.187086820602417 4450.226204872131
20020/20434
Houston Rockets Atlanta Hawks 0.12361717224121094 4450.350157737732
20021/20434
Denver Nuggets Dallas Mavericks 0.08466196060180664 4450.437703847885
20022/20434
Chicago Bulls New Orleans Pelicans 0.08516979217529297 4450.524197816849
20023/20434

Los Angeles Lakers Boston Celtics 0.18164610862731934 4462.057974100113
20111/20434
Denver Nuggets Minnesota Timberwolves 0.21765494346618652 4462.276169061661
20112/20434
San Antonio Spurs Phoenix Suns 0.11957573890686035 4462.396101951599
20113/20434
Indiana Pacers Utah Jazz 0.1143341064453125 4462.511004924774
20114/20434
Golden State Warriors Portland Trail Blazers 0.20374608039855957 4462.714949846268
20115/20434
Los Angeles Clippers Dallas Mavericks 0.11859989166259766 4462.834707021713
20116/20434
Sacramento Kings Detroit Pistons 0.08426070213317871 4462.919162988663
20117/20434
Oklahoma City Thunder Orlando Magic 0.11790013313293457 4463.038076877594
20118/20434
Philadelphia 76ers Toronto Raptors 0.10122299194335938 4463.139789104462
20119/20434
Los Angeles Clippers Atlanta Hawks 0.15326404571533203 4463.293596982956
20120/20434
Memphis Grizzlies Boston Celtics 0.09167170524597168 4463.385530948639
20121/20434
Washington Wizards Miami Heat 0.11174392700195312 4463.4975707530975

San Antonio Spurs Los Angeles Clippers 0.08627462387084961 4475.571995973587
20212/20434
Milwaukee Bucks New Orleans Pelicans 0.16584491729736328 4475.738137960434
20213/20434
Charlotte Hornets Houston Rockets 0.10939216613769531 4475.847774982452
20214/20434
Portland Trail Blazers Denver Nuggets 0.08782815933227539 4475.936984062195
20215/20434
San Antonio Spurs Los Angeles Lakers 0.10595512390136719 4476.0431706905365
20216/20434
Phoenix Suns Detroit Pistons 0.1246039867401123 4476.168048858643
20217/20434
Orlando Magic Boston Celtics 0.1589829921722412 4476.328560829163
20218/20434
Golden State Warriors Brooklyn Nets 0.16427898406982422 4476.494120836258
20219/20434
Indiana Pacers Toronto Raptors 0.13268089294433594 4476.628233909607
20220/20434
Atlanta Hawks Minnesota Timberwolves 0.21061420440673828 4476.839119911194
20221/20434
Cleveland Cavaliers Oklahoma City Thunder 0.1018831729888916 4476.941268920898
20222/20434
Memphis Grizzlies Dallas Mavericks 0.15514206886291504 4477.097

Miami Heat Cleveland Cavaliers 0.16943812370300293 4488.78213596344
20311/20434
Atlanta Hawks Philadelphia 76ers 0.09498190879821777 4488.8774700164795
20312/20434
Milwaukee Bucks Washington Wizards 0.1014108657836914 4488.97917175293
20313/20434
Orlando Magic Brooklyn Nets 0.23119521141052246 4489.210808038712
20314/20434
New York Knicks Houston Rockets 0.17079496383666992 4489.382877826691
20315/20434
Minnesota Timberwolves Dallas Mavericks 0.13303184509277344 4489.516211032867
20316/20434
Phoenix Suns Utah Jazz 0.10116291046142578 4489.618020772934
20317/20434
Memphis Grizzlies Los Angeles Clippers 0.16907405853271484 4489.78774189949
20318/20434
Charlotte Hornets Indiana Pacers 0.09047412872314453 4489.878531932831
20319/20434
Milwaukee Bucks Toronto Raptors 0.0940849781036377 4489.973016738892
20320/20434
Oklahoma City Thunder Chicago Bulls 0.1186678409576416 4490.091902971268
20321/20434
Detroit Pistons Denver Nuggets 0.15214109420776367 4490.244433879852
20322/20434
New Orleans 

Los Angeles Lakers Los Angeles Clippers 0.19805288314819336 4502.873903751373
20410/20434
Oklahoma City Thunder Boston Celtics 0.10263514518737793 4502.977219820023
20411/20434
Milwaukee Bucks Phoenix Suns 0.11105895042419434 4503.088844060898
20412/20434
Miami Heat Washington Wizards 0.24066996574401855 4503.330293893814
20413/20434
Indiana Pacers Dallas Mavericks 0.14498305320739746 4503.47615480423
20414/20434
Orlando Magic Houston Rockets 0.16158819198608398 4503.638818740845
20415/20434
Detroit Pistons New York Knicks 0.16993093490600586 4503.808985948563
20416/20434
San Antonio Spurs Cleveland Cavaliers 0.10692095756530762 4503.916141033173
20417/20434
Toronto Raptors Sacramento Kings 0.127000093460083 4504.0434629917145
20418/20434
Charlotte Hornets Atlanta Hawks 0.30098700523376465 4504.344793796539
20419/20434
Milwaukee Bucks Denver Nuggets 0.1392381191253662 4504.484290122986
20420/20434
Toronto Raptors Utah Jazz 0.15809321403503418 4504.642771959305
20421/20434
New York Knic

In [17]:
testOutcomeData.to_csv('/Users/lorneez/projects/sports_predictor/Sports_Model/Data/feature_data.csv')
testOutcomeData

,FGA_PG_home,FGA_PG_visitor,FG%_PG_home,FG%_PG_visitor,3P_PG_home,3P_PG_visitor,3PA_PG_home,3PA_PG_visitor,3P%_PG_home,3P%_PG_visitor,...,TRB_PG_home,TRB_PG_visitor,AST_PG_home,AST_PG_visitor,STL_PG_home,STL_PG_visitor,BLK_PG_home,BLK_PG_visitor,TOV_PG_home,TOV_PG_visitor
0,"23 78.3 Name: FGA_PG, dtype: float64","19 78.3 Name: FGA_PG, dtype: float64","23 0.444 Name: FG%_PG, dtype: float64","19 0.443 Name: FG%_PG, dtype: float64","23 4.4 Name: 3P_PG, dtype: float64","19 6.7 Name: 3P_PG, dtype: float64","23 12.8 Name: 3PA_PG, dtype: float64","19 18.5 Name: 3PA_PG, dtype: float64","23 0.345 Name: 3P%_PG, dtype: float64","19 0.364 Name: 3P%_PG, dtype: float64",...,"23 43.4 Name: TRB_PG, dtype: float64","19 42.4 Name: TRB_PG, dtype: float64","23 21.8 Name: AST_PG, dtype: float64","19 21.1 Name: AST_PG, dtype: float64","23 7.0 Name: STL_PG, dtype: float64","19 6.9 Name: STL_PG, dtype: float64","23 6.1 Name: BLK_PG, dtype: float64","19 4.6 Name: BLK_PG, dtype: float64","23 13.8 Name: TOV_PG, dtype: float64","19 13.8 Name: TOV_PG, dtype: float64"
1,"2 81.6 Name: FGA_PG, dtype: float64","1 85.1 Name: FGA_PG, dtype: float64","2 0.457 Name: FG%_PG, dtype: float64","1 0.459 Name: FG%_PG, dtype: float64","2 5.6 Name: 3P_PG, dtype: float64","1 6.4 Name: 3P_PG, dtype: float64","2 15.5 Name: 3PA_PG, dtype: float64","1 17.0 Name: 3PA_PG, dtype: float64","2 0.364 Name: 3P%_PG, dtype: float64","1 0.374 Name: 3P%_PG, dtype: float64",...,"2 42.9 Name: TRB_PG, dtype: float64","1 42.4 Name: TRB_PG, dtype: float64","2 19.6 Name: AST_PG, dtype: float64","1 24.5 Name: AST_PG, dtype: float64","2 8.6 Name: STL_PG, dtype: float64","1 8.2 Name: STL_PG, dtype: float64","2 5.6 Name: BLK_PG, dtype: float64","1 3.9 Name: BLK_PG, dtype: float64","2 13.4 Name: TOV_PG, dtype: float64","1 13.1 Name: TOV_PG, dtype: float64"
2,"11 80.8 Name: FGA_PG, dtype: float64","7 80.7 Name: FGA_PG, dtype: float64","11 0.437 Name: FG%_PG, dtype: float64","7 0.459 Name: FG%_PG, dtype: float64","11 7.9 Name: 3P_PG, dtype: float64","7 3.9 Name: 3P_PG, dtype: float64","11 22.1 Name: 3PA_PG, dtype: float64","7 11.5 Name: 3PA_PG, dtype: float64","11 0.355 Name: 3P%_PG, dtype: float64","7 0.34 Name: 3P%_PG, dtype: float64",...,"11 43.2 Name: TRB_PG, dtype: float64","7 42.0 Name: TRB_PG, dtype: float64","11 20.4 Name: AST_PG, dtype: float64","7 23.9 Name: AST_PG, dtype: float64","11 6.2 Name: STL_PG, dtype: float64","7 9.1 Name: STL_PG, dtype: float64","11 4.2 Name: BLK_PG, dtype: float64","7 6.0 Name: BLK_PG, dtype: float64","11 14.3 Name: TOV_PG, dtype: float64","7 14.9 Name: TOV_PG, dtype: float64"
3,"16 81.5 Name: FGA_PG, dtype: float64","24 75.2 Name: FGA_PG, dtype: float64","16 0.447 Name: FG%_PG, dtype: float64","24 0.432 Name: FG%_PG, dtype: float64","16 3.7 Name: 3P_PG, dtype: float64","24 6.6 Name: 3P_PG, dtype: float64","16 11.0 Name: 3PA_PG, dtype: float64","24 19.2 Name: 3PA_PG, dtype: float64","16 0.332 Name: 3P%_PG, dtype: float64","24 0.344 Name: 3P%_PG, dtype: float64",...,"16 42.3 Name: TRB_PG, dtype: float64","24 40.1 Name: TRB_PG, dtype: float64","16 22.6 Name: AST_PG, dtype: float64","24 18.2 Name: AST_PG, dtype: float64","16 8.0 Name: STL_PG, dtype: float64","24 7.5 Name: STL_PG, dtype: float64","16 5.6 Name: BLK_PG, dtype: float64","24 4.4 Name: BLK_PG, dtype: float64","16 13.9 Name: TOV_PG, dtype: float64","24 14.3 Name: TOV_PG, dtype: float64"
4,"8 81.5 Name: FGA_PG, dtype: float64","14 80.2 Name: FGA_PG, dtype: float64","8 0.454 Name: FG%_PG, dtype: float64","14 0.45 Name: FG%_PG, dtype: float64","8 3.9 Name: 3P_PG, dtype: float64","14 4.0 Name: 3P_PG, dtype: float64","8 11.2 Name: 3PA_PG, dtype: float64","14 11.3 Name: 3PA_PG, dtype: float64","8 0.349 Name: 3P%_PG, dtype: float64","14 0.351 Name: 3P%_PG, dtype: float64",...,"8 43.7 Name: TRB_PG, dtype: float64","14 41.3 Name: TRB_PG, dtype: float64","8 19.3 Name: AST_PG, dtype: float64","14 21.0 Name: AST_PG, dtype: float64","8 7.7 Name: STL_PG, dtype: float64","14 6.5 Name: STL_PG, dtype: float64","8 5.4 Name

In [ ]:
x_columns = []
for col in teamDataColumn:
    x_columns.append(col + "_DIFF")
print(x_columns)

### Training Model

In [13]:
# target column (in this case, whether or not the home team will win)
y = outcomeData["HomeWin"]
# y = outcomeData["Final Home Spread"]

# columns that will be used to make the prediction
# X = outcomeData[x_columns]
# X = testOutcomeData
X_previous_wins = outcomeData[["HomeLastWin", "VisitorLastWin"]].values
X_win_streak = outcomeData[["HomeLastWin", "VisitorLastWin", "HomeWinStreak", "VisitorWinStreak"]].values

# X = outcomeData[["HomeLastWin", "VisitorLastWin", "HomeTeamRanksHigher", "HomeTeamWonLast"]].values

# set up scorer for testing accuracy
scorer = make_scorer(f1_score, pos_label = None, average = "weighted")

In [14]:
# Set up train test split
# X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(X_1, y, test_size=0.20, random_state=100)
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=100)

In [15]:
y_true = outcomeData["HomeWin"].values

# cross val score
clf = RandomForestClassifier(random_state = 14)
#scores = cross_val_score(clf, X_train, y_train, scoring=scorer)
scores_previous_wins = cross_val_score(clf, X_previous_wins, y_true, scoring=scorer)
scores_win_streak = cross_val_score(clf, X_win_streak, y_true, scoring=scorer)

In [16]:
# mean score
y_pred = [1] * len(y_true)

print("F1 Score to Beat: {:.4f}".format(f1_score(y_true, y_pred, pos_label=None, average='weighted')))

print("F1 Score with Previous Wins: {:.4f}".format(np.mean(scores_previous_wins)))

print("F1 Score with Win Streaks: {:.4f}".format(np.mean(scores_win_streak)))


F1 Score to Beat: 0.4434
F1 Score with Previous Wins: 0.4434
F1 Score with Win Streaks: 0.5128
